In [1]:
import os
import glob
import json
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 12

# get location of pf-ci json files
#pfci_file_location = os.path.join('/Users/rmandern/code/data_repository/Mapol/OHminus', 'qed-fci', '*.json')
#pcqed_file_location = os.path.join('/Users/rmandern/code/data_repository/Mapol/OHminus', 'pcqed_data', '*.json')
pfci_file_location = os.path.join('/Users/jfoley19/Code/data_repository/Mapol/OHminus', 'qed-fci', '*.json')
pcqed_file_location = os.path.join('/Users/jfoley19/Code/data_repository/Mapol/OHminus', 'pcqed_data', '*.json')
# get pf-ci file names
pfci_filenames = glob.glob(pfci_file_location)
pcqed_filenames = glob.glob(pcqed_file_location)

# get number of the pf-ci json files
n_files = len(pfci_filenames)

n_files_2 = len(pcqed_filenames)

print(pcqed_filenames[0])
print(pcqed_filenames[1])

pcqed_data_origin = json.load(open(pcqed_filenames[1]))
pcqed_data_displaced = json.load(open(pcqed_filenames[0]))



/Users/jfoley19/Code/data_repository/Mapol/OHminus/pcqed_data/OH-_6-31G_PCQED_Origin_Dependence_20_ang_0.21927747.json
/Users/jfoley19/Code/data_repository/Mapol/OHminus/pcqed_data/OH-_6-31G_PCQED_Origin_Dependence_0_ang_0.21927747.json


In [6]:
photon_count = pcqed_data_origin["Photon_list"]

print(photon_count)
print(pcqed_data_origin["return_result"][0][0])
print(pcqed_data_origin["return_result"][0][1])
print(pcqed_data_origin["return_result"][1][0])

[3, 5, 7, 9, 11, 13, 15, 17, 33]
-75.43266530871836
-75.23588817286917
-75.43266533500551


In [ ]:

# create numpy array to hold the number of photonic fock states
pn = np.zeros(n_files)
# create numpy array to hold the displacement error
er = np.zeros(n_files)

# loop over json files
ctr = 0
for f in pfci_filenames:
    # load json file
    data = json.load(open(f))
    # get data field with number of photonic fock states
    n = data["model"]["number_of_photon_states"]
    # get data field with origin dependent error
    error = data["origin_dependent_error"]
    # store n to numpy array
    pn[ctr] = n
    # store error to numpy array
    er[ctr] = error
    # increment counter
    ctr += 1

# find indices sorted from lowest to highest photonic fock state numbers
idx = np.flip(np.argsort(pn))


x_pfci = pn[idx]
y_pfci = er[idx]


N_p = np.array([3, 5, 7, 9, 11, 13, 15, 17, 33])
error = np.array([0.8846499098818299,0.5030561682658288,0.28350180325411145,0.15179584422928372,0.0744678498403033,0.03215178125074658,0.011680933216794642,0.0034267888430434823,1.298161578233703e-10])
# N_pc = N_p[idx]
# err = error[idx]

coef_pfci = np.polyfit(x_pfci, y_pfci, 5)

coef_pcqed = np.polyfit(N_p, error, 5)

def poly_eval(x_vals, a_vals, order):
    """ Function to evaluate a polynomial  or order 2, 3, or 4 from np.polyfit
        Arguments
        ---------
        x_vals : array of floats
            the x values you want to evaluate the fit on
        a_vals : array of floats
            the coefficient of the polynomial
        order : int
            the order of the polynomial - accepts 2, 3, or 4

    """
    if order==5:
        return a_vals[0] * x_vals ** 5 + a_vals[1] * x_vals ** 4 + a_vals[2] * x_vals ** 3 + a_vals[3] * x_vals ** 2 + a_vals[4] * x_vals ** 1 + a_vals[5]
    elif order==4:
        return a_vals[0] * x_vals ** 4 + a_vals[1] * x_vals ** 3 + a_vals[2] * x_vals ** 2 + a_vals[3] * x_vals ** 1 + a_vals[4]
    elif order==3:
        return a_vals[0] * x_vals ** 3 + a_vals[1] * x_vals ** 2 + a_vals[2] * x_vals ** 1 + a_vals[3]

    elif order==2:
        return a_vals[0] * x_vals ** 2 + a_vals[1] * x_vals ** 1 + a_vals[2] 
    else:
        print(" Specify order = 2, 3, or 4")
        exit()
        

In [ ]:
print(x_pfci)
print(y_pfci)

print(N_p)
print(error)

In [ ]:
# pfit_pcqed = np.poly1d(pcqed_pfit)
# x_val = np.linspace(min(N_p), max(N_p), 9)
# y_val = pfit_pcqed(x_val)


COLOUR1 = "firebrick"
COLOUR2 = "darkgreen"
COLOUR3 = "royalblue"
COLOUR4 = "rebeccapurple"
COLOUR5 = 'darkorchid'
COLOUR6 = 'olivedrab'

fit_pfci = poly_eval(x_pfci, coef_pfci, 5)

fit_pcqed = poly_eval(N_p, coef_pcqed, 5)

plt.plot(x_pfci, y_pfci, color=COLOUR1, label="PF-FCI-N/6-31G" )
plt.plot(N_p, error, color=COLOUR2, label="pPF" )
plt.plot(x_pfci, fit_pfci, color=COLOUR1, linestyle="none", marker="o",  ms="8", mfc="none", label="Polynomial Fit PF-FCI")
plt.plot(N_p, fit_pcqed, color=COLOUR2, linestyle="none", marker="o",  ms="8", mfc="none", label="Polynomial Fit pPF")
# plt.yscale("log")
plt.legend()
plt.xlabel("Number of Photonic Fock States")
plt.ylabel("Origin Dependent Error")
plt.savefig("OH_minus_error_origin_dep.png", dpi=150)

plt.show()
